In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
!unzip -q "/content/resume_data.zip" -d "/content/"


In [ ]:
import os
for dirname, _, filenames in os.walk('/content/data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/data/data/DIGITAL-MEDIA/91539554.pdf
/content/data/data/DIGITAL-MEDIA/18354623.pdf
/content/data/data/DIGITAL-MEDIA/17432318.pdf
/content/data/data/DIGITAL-MEDIA/62700506.pdf
/content/data/data/DIGITAL-MEDIA/34319869.pdf
/content/data/data/DIGITAL-MEDIA/20330739.pdf
/content/data/data/DIGITAL-MEDIA/21125113.pdf
/content/data/data/DIGITAL-MEDIA/18900722.pdf
/content/data/data/DIGITAL-MEDIA/19861776.pdf
/content/data/data/DIGITAL-MEDIA/25723793.pdf
/content/data/data/DIGITAL-MEDIA/17562754.pdf
/content/data/data/DIGITAL-MEDIA/24658786.pdf
/content/data/data/DIGITAL-MEDIA/15226699.pdf
/content/data/data/DIGITAL-MEDIA/58915642.pdf
/content/data/data/DIGITAL-MEDIA/34583750.pdf
/content/data/data/DIGITAL-MEDIA/18488289.pdf
/content/data/data/DIGITAL-MEDIA/26160200.pdf
/content/data/data/DIGITAL-MEDIA/25706337.pdf
/content/data/data/DIGITAL-MEDIA/19053815.pdf
/content/data/data/DIGITAL-MEDIA/27419236.pdf
/content/data/data/DIGITAL-MEDIA/18525641.pdf
/content/data/data/DIGITAL-MEDIA/2

In [ ]:
resume_data = pd.read_csv("/content/Resume/Resume.csv")
resume_data

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR
...,...,...,...,...
2479,99416532,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
2480,24589765,"GOVERNMENT RELATIONS, COMMUNICATIONS ...","<div class=""fontsize fontface vmargins hmargin...",AVIATION
2481,31605080,GEEK SQUAD AGENT Professional...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
2482,21190805,PROGRAM DIRECTOR / OFFICE MANAGER ...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION


In [ ]:
pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 5.2 MB/s eta 0:00:00


In [ ]:
from pypdf import PdfReader
def extract_text_from_pdf(file_path):
    reader = PdfReader(file_path)
    text = "".join(page.extract_text() for page in reader.pages)
    return text

In [ ]:
from nltk import pos_tag, sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import string
import re
def preprocess_text(text):
    text = text.lower()
    text = re.sub('[^a-zA-Z]', ' ', text)
    sentences = sent_tokenize(text)
    features = {'feature': ""}
    stop_words = set(stopwords.words("english"))
    for sent in sentences:
        if any(criteria in sent for criteria in ['skills', 'education']):
            words = word_tokenize(sent)
            words = [word for word in words if word not in stop_words]
            tagged_words = pos_tag(words)
            filtered_words = [word for word, tag in tagged_words if tag not in ['DT', 'IN', 'TO', 'PRP', 'WP']]
            features['feature'] += " ".join(filtered_words)
    return features

In [ ]:
def process_resume_data(df):
    id = df['ID']
    category = df['Category']
    text = extract_text_from_pdf(f"/content/data/data/{category}/{id}.pdf")
    features = preprocess_text(text)
    df['Feature'] = features['feature']
    return df

In [ ]:

num_desc = 3
resume_data = pd.read_csv("/content/Resume/Resume.csv")
resume_data = resume_data.drop(["Resume_html"], axis=1)
resume_data = resume_data.apply(process_resume_data, axis=1)
resume_data = resume_data.drop(columns=['Resume_str'])
resume_data.to_csv("/content/resume_data.csv", index=False)
job_description = pd.read_csv("/content/training_data.csv")
job_description = job_description[["job_description", "position_title"]][:num_desc]
job_description['Features'] = job_description['job_description'].apply(lambda x: preprocess_text(x)['feature'])


In [ ]:
resume_data

,ID,Category,Feature
0,16852973,HR,hr administrator marketing associate hr admini...
1,22323967,HR,hr specialist hr operations summary media prof...
2,33176873,HR,hr director summary years experience recruitin...
3,27018550,HR,hr specialist summary dedicated driven dynamic...
4,17812897,HR,hr manager skill highlights hr skills hr depar...
...,...,...,...
2479,99416532,AVIATION,rank sgt e non commissioned officer charge bri...
2480,24589765,AVIATION,government relations communications organizati...
2481,31605080,AVIATION,geek squad agent professional profile support ...
2482,21190805,AVIATION,program director office manager summary highly...


In [ ]:
categories = np.sort(resume_data['Category'].unique())
categories
# create new df for corpus and category
df_categories = [resume_data[resume_data['Category'] == category].loc[:, ['Feature', 'Category']] for category in categories]

In [ ]:
def remove_extra_word(text):

    extra_word=['company', 'name', 'citi', 'state', 'work', 'manag', 'project'] # extra words
    words = text.split()
    filter_word = [word for word in words if word not in extra_word]
    filter_text = ' '.join(filter_word)
    return filter_text
resume_data['Feature']=resume_data['Feature'].apply(lambda x:remove_extra_word(x))

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

resume_data['Category']=encoder.fit_transform(resume_data['Category'])
resume_data.head()

,ID,Category,Feature
0,16852973,19,hr administrator marketing associate hr admini...
1,22323967,19,hr specialist hr operations summary media prof...
2,33176873,19,hr director summary years experience recruitin...
3,27018550,19,hr specialist summary dedicated driven dynamic...
4,17812897,19,hr manager skill highlights hr skills hr depar...


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(resume_data['Feature'], resume_data['Category'], test_size=0.15, random_state=42, stratify=resume_data['Category'])


# Print the sizes of the split datasets
print("Train data size:", X_train.shape)
print("Validation data size:", X_test.shape)

Train data size: (2111,)
Validation data size: (373,)


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize and pad the sequences
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)


max_length = 100
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length, padding='post', truncating='post')


In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Set up a custom Adam optimizer with a lower learning rate
optimizer = Adam(learning_rate=0.0005)  # Try values like 0.001, 0.0005, or 0.0001

# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=3,          # Number of epochs to wait for improvement
    restore_best_weights=True
)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

# Define the model
model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=max_length),  # Increased embedding dimension
    Bidirectional(LSTM(128, return_sequences=True)),  # Increased LSTM units
    Dropout(0.3),  # Increased dropout rate
    Bidirectional(LSTM(128, return_sequences=True)),
    Dropout(0.3),
    LSTM(64),
    Dense(64, activation='relu'),  # Added dense layer
    Dropout(0.3),
    Dense(32, activation='relu'),  # Another dense layer for additional depth
    Dropout(0.3),
    Dense(len(categories), activation='softmax')  # Output layer
])

# Compile the model with a specified optimizer
optimizer = Adam(learning_rate=0.001)  # Set learning rate
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Print model summary
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_5 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_6 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_12 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_15 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_16 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:

history = model.fit(
    X_train_padded, y_train,
    validation_data=(X_test_padded, y_test),
    epochs=50,
    batch_size=32,
    verbose=2
)


Epoch 1/50
66/66 - 83s - 1s/step - accuracy: 0.0483 - loss: 3.1477 - val_accuracy: 0.0804 - val_loss: 3.0336
Epoch 2/50
66/66 - 67s - 1s/step - accuracy: 0.0815 - loss: 3.0057 - val_accuracy: 0.1206 - val_loss: 2.9563
Epoch 3/50
66/66 - 87s - 1s/step - accuracy: 0.1085 - loss: 2.8231 - val_accuracy: 0.1206 - val_loss: 3.0261
Epoch 4/50
66/66 - 80s - 1s/step - accuracy: 0.1360 - loss: 2.7396 - val_accuracy: 0.1180 - val_loss: 2.8597
Epoch 5/50
66/66 - 83s - 1s/step - accuracy: 0.1649 - loss: 2.5491 - val_accuracy: 0.1582 - val_loss: 2.8019
Epoch 6/50
66/66 - 81s - 1s/step - accuracy: 0.1909 - loss: 2.3817 - val_accuracy: 0.1796 - val_loss: 2.7454
Epoch 7/50
66/66 - 70s - 1s/step - accuracy: 0.2032 - loss: 2.3496 - val_accuracy: 0.1421 - val_loss: 2.9552
Epoch 8/50
66/66 - 83s - 1s/step - accuracy: 0.2459 - loss: 2.1915 - val_accuracy: 0.1743 - val_loss: 2.7606
Epoch 9/50
66/66 - 80s - 1s/step - accuracy: 0.2681 - loss: 2.0387 - val_accuracy: 0.1555 - val_loss: 3.0013
Epoch 10/50
66/66 -

In [ ]:
# Save the model as an HDF5 file
model.save("/content/resume_model.h5")


In [ ]:
model.save('my_model.keras')

In [ ]:
!pip install tensorflowjs


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 48.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninstalled packaging-24.1


In [ ]:
import tensorflowjs as tfjs

# Convert and save the model in TensorFlow.js format
tfjs.converters.save_keras_model(model, "/content/tfjs_resume_model")


failed to lookup keras version from the file,
    this is likely a weight only file
